In [254]:
import requests
from bs4 import BeautifulSoup

# URL = 'https://www.macrotrends.net/stocks/charts/DELL/dell/free-cash-flow'
URL = "https://www.macrotrends.net/stocks/charts/DIS/disney/free-cash-flow"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [255]:
cashflows = {}

tbody = soup.find("table", "historical_data_table").find("tbody")
for tr in reversed(tbody.findAll("tr")):
    trs = tr.findAll("td")
    year = int(trs[0].text.split("-")[0])
    if year > 2018: continue # eliminate post-covid
    price = float(trs[1].text.replace(",","").strip())
    print(year, "    ", price)
    cashflows[year] = price

2009      3566.0
2010      4468.0
2011      3435.0
2012      4182.0
2013      6656.0
2014      6469.0
2015      7120.0
2016      8363.0
2017      8720.0
2018      9830.0


In [256]:
import pandas as pd

df = pd.DataFrame([cashflows])
df

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,3566.0,4468.0,3435.0,4182.0,6656.0,6469.0,7120.0,8363.0,8720.0,9830.0


In [257]:
growth_rates = {}

for year1, year2 in zip(df, df.iloc[:, 1:]):
    price1 = df[year1].iloc[0]
    price2 = df[year2].iloc[0]
    growth_rate = (price2 - price1) / price1
    growth_rates[year2] = growth_rate

In [258]:
df = df.append(growth_rates, ignore_index=True)
df

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,3566.0,4468.000000,3435.0000,4182.000000,6656.000000,6469.000000,7120.000000,8363.000000,8720.000000,9830.000000
1,NaN,0.252944,-0.2312,0.217467,0.591583,-0.028095,0.100634,0.174579,0.042688,0.127294


In [259]:
import numpy as np

avg_growth_rate = np.mean(list(growth_rates.values()))
df.iloc[1].mean()

0.13865490639469857

In [260]:
future_cashflows = {}
future_cashflows[2023] = round(df.iloc[:, -1].iloc[0])
for year in range(2024, 2031):
    future_cashflows[year] = round(future_cashflows[year-1] + (future_cashflows[year-1] * avg_growth_rate))

future_cashflows

{2023: 9830,
 2024: 11193,
 2025: 12745,
 2026: 14512,
 2027: 16524,
 2028: 18815,
 2029: 21424,
 2030: 24395}

In [261]:
future_df = pd.DataFrame([future_cashflows])
future_df

,2023,2024,2025,2026,2027,2028,2029,2030
0,9830,11193,12745,14512,16524,18815,21424,24395


In [262]:
# rate at which the company will grow in the future. Usually 2.5%-3% as economy moves in that pace
perpetual_growth_rate = 0.025
# how much cheaper we want it to get
dicount_rate = 0.08

In [263]:
price_2030 = future_df[2030].iloc[0]
terminal_value = round(price_2030 * (1+perpetual_growth_rate) / (dicount_rate-perpetual_growth_rate))

In [264]:
terminal_value

454634

In [265]:
future_df["Terminal"] = terminal_value
future_df

,2023,2024,2025,2026,2027,2028,2029,2030,Terminal
0,9830,11193,12745,14512,16524,18815,21424,24395,454634


In [266]:
PV_of_FFCFs = {}
for (i, col) in enumerate(future_df):
    PV_of_FFCFs[col] = round(future_df[col].iloc[0] / (1 + dicount_rate)**(i+1))
PV_of_FFCFs

{2023: 9102,
 2024: 9596,
 2025: 10117,
 2026: 10667,
 2027: 11246,
 2028: 11857,
 2029: 12501,
 2030: 13180,
 'Terminal': 227430}

In [267]:
future_df = future_df.append(PV_of_FFCFs, ignore_index=True)
future_df

,2023,2024,2025,2026,2027,2028,2029,2030,Terminal
0,9830,11193,12745,14512,16524,18815,21424,24395,454634
1,9102,9596,10117,10667,11246,11857,12501,13180,227430


In [268]:
sum_of_FCF = future_df.iloc[1].sum()
sum_of_FCF

315696

In [269]:
URL = "https://au.finance.yahoo.com/quote/DIS/balance-sheet?p=DIS"
page = requests.get(URL, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0'})

soup = BeautifulSoup(page.content, "html.parser")
cash = soup.find("div", {"title": "Cash and cash equivalents"})
cash_and_eq = int(cash.next.next.next.next.span.text.replace(",", ""))/1000
cash_and_eq

15959.0

In [270]:
def convert_to_B(val):
    if (val[-1] == "B"):
        val = val.replace("B", "")
        while (len(val.split(".")[-1]) < 3):
            val += "0"
        val = int(val.replace(".", ""))
        return val
    elif (val[-1] == "M"):
        val = val.replace(".", "").replace("M", "")
        val = float("0." + val)
        return val

In [271]:
URL = "https://uk.finance.yahoo.com/quote/DIS/key-statistics?p=DIS"
page = requests.get(URL, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0'})

soup = BeautifulSoup(page.content, "html.parser")
total_debt = soup.findAll("table", {"class": "Bdcl(c)"})[8].findAll("tr")[2].findAll("td")[1].text
# while (len(total_debt.split(".")[-1]) < 3):
#     total_debt += "0"
# total_debt = int(total_debt.replace(".", ""))
total_debt = convert_to_B(total_debt)
total_debt

52020

In [272]:
equity_value = sum_of_FCF + cash_and_eq - total_debt
equity_value

279635.0

In [273]:
shares_outstanding = soup.findAll("table", {"class": "Bdcl(c)"})[2].findAll("tr")[2].findAll("td")[1].text
shares_outstanding = convert_to_B(shares_outstanding)
shares_outstanding

1820

In [274]:
price_per_share = equity_value / shares_outstanding
price_per_share

153.6456043956044